In [3]:
%pip install nbformat
import nltk
nltk.download('omw-1.4')

%run "Data_Handling/Data_Preprocessing_AILA.ipynb"

Note: you may need to restart the kernel to use updated packages.


[nltk_data] Downloading package omw-1.4 to /home/hamna/nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!


Note: you may need to restart the kernel to use updated packages.
"Arumugam v State, Represented By Inspector of Police, Tamil Nadu Supreme Court of India  13 October 2008 Criminal Appeal No. 967 of 2001 The Judgment was delivered by : Dr. Arijit Pasayat, J. 1.  Challenge in this appeal is to the judgment of the learned Single Judge of the Madras High Court upholding the conviction of the appellants for the offence punishable under Section 302 of the Indian Penal Code, 1860 (in short 'IPC') so far accused A1 is concerned, and the co-accused for the offence punishable under Section 302 read with Section 34 IPC. Learned District and Sessions Judge, Tirunvelveli, has held the appellant and the co-accused guilty and as aforenoted by judgment dated 20.10.1989 in Sessions Case No.140 of 1987. Another person Subbiah-A3 was charged for commission of offence punishable under Section 323 IPC. A3 did not prefer appeal before the High Court and A2 has also not filed any appeal before this Court. B

[nltk_data] Downloading package stopwords to /home/hamna/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to /home/hamna/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


{'giddy', 'madh', 'cation', '4298', 'dharmu', '31121995', 'officersrdos', 'meps', 'relief', 'intuc', 'mahabatra', 'aswell', 'advocateonrecord', 'extka8', '14681', 'santhi', '22372238', 'banco', 'palachuru', 'sardana', '10111987', 'idris', '11x2x1', '1982005', 'ayacutthe', 'pvr', 'kuria', '29121998', 'ventilating', 'gairmaurusiawal', 'no807', 'unmistakebly', 'surveynumber', 'identifiedset', 'stirring', 'sensed', 'raghutilaka', 'pichey', 'chinthagumpala', 'inam', 'vrreddy', 'orswith', '694', 'sukhe', 'venkteswarlu', 'occupationalauthority', '4073', 'dadji', '7395', 'midauxiliary', 'spondentcompany', 'bhaj', '58232', '2651945', 'accident', 'austin', 'accompaniment', 'safekeeping', 'mansukhlal', '4532', 'subsecs', 'brought', 'munsif1', 'microorganism', 'nikalna', 'booster', 'radhabai', 'blood', 'mahjiladih', 'valujaraja', '4486', 'bhairebendra', '3311261', 'mzq', 'gulabi', 'relieving', 'cheddu', 'lingareddy', '5311959', '1246', 'vy', 'saurashtra', 'improbability', 'reflected', '1631979', '

Executing Data_Preprocessing.ipynb to use the variables and functions in it.

In [12]:
documents_df.head(5)

,Document_Name,Document_Text,preprocessed_text
0,C2543.txt,"Arumugam v State, Represented By Inspector of ...",arumugam v state represented inspector police ...
1,C1557.txt,"Hindu Jea Band, Jaipur v Regional Director, Em...",hindu jea band jaipur v regional director empl...
2,C2457.txt,Mohd. Rashid Ahmad and another v State Of U.P....,mohd rashid ahmad another v state anr supreme ...
3,C1264.txt,Bhagirath and Others v State of Haryana Suprem...,bhagirath others v state haryana supreme court...
4,C1374.txt,Animireddy Venkata Ramana and Others v Public ...,animireddy venkata ramana others v public pros...


In [5]:
pd.DataFrame(queries_df["preprocessed_text"].head())

,preprocessed_text
0,appellant february 9 1961 appointed officer gr...
1,appellant u examined prime witness trial tr fi...
2,appeal arises judgment learned single judge hi...
3,petitioner married respondent no2 27th novembe...
4,appeal preferred judgment dated 1982011 passed...


In [6]:
query_document_relevance_pairings.head(5)

,Query_Name,Document_Name,Relevance
0,AILA_Q1,C168,0
1,AILA_Q1,C382,0
2,AILA_Q1,C428,0
3,AILA_Q1,C949,0
4,AILA_Q1,C2303,0


In [7]:
# Inverted Index Calculation

def calculate_inverted_index(text):
    inverted_index = {}
    for doc_id in range(len(text)):
        tokens = text[doc_id]
        for token in tokens:
            if token not in inverted_index:
                inverted_index[token] = set()
            inverted_index[token].add(doc_id)
    return inverted_index

inverted_index = calculate_inverted_index(documents_df["preprocessed_text"])
# for key, value in inverted_index.items():
#     print(f'{key}: {value}')

## Vector Space Model

In [8]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
import numpy as np

In [9]:
tf_idf_Vectorizer = TfidfVectorizer(min_df=1)
tf_idf_documents = tf_idf_Vectorizer.fit_transform(documents_df["preprocessed_text"])
print(tf_idf_documents.toarray().shape)

# print(tf_idf_query.toarray().shape)

(2914, 97588)


Vectorizes each of the document and the query with tf-idf weights using TfidfVectorizer. These vectors are used to compute cosine similarity. Here we have used a single query from queries_df dataframe.

In [23]:
precision_scores = []
recall_scores = []

In [24]:
for index, row in queries_df.iterrows():
    query = row['preprocessed_text']
    query_id = row['Query_Name']
    query_relevance_data = query_document_relevance_pairings[query_document_relevance_pairings['Query_Name'] == query_id]
    tf_idf_query = tf_idf_Vectorizer.transform([query])
    cosine_similarities = cosine_similarity(tf_idf_query, tf_idf_documents)
    top_10_indices = np.argsort(cosine_similarities[0])[-20:][::-1]
    top_k_docs = []
    for i, index in enumerate(top_10_indices):
        top_k_docs.append("C" + str(index+1))
    # print(top_k_docs)
    actual_relevant_docs = query_relevance_data[query_relevance_data['Relevance'] == 1]['Document_Name'].tolist()
    # print(actual_relevant_docs)
    num_actual = 0
    for doc in top_k_docs:
        if doc in actual_relevant_docs:
            num_actual += 1
    precision = num_actual/len(top_k_docs)
    recall = num_actual/len(actual_relevant_docs)
    if precision > 0:
        print(num_actual)
    precision_scores.append(precision)
    recall_scores.append(recall)



1


Calculates the cosine similarity between each document and the query. Returns an array of all similarity values in order of document ids.

In [25]:
avg_precision = sum(precision_scores) / 50
avg_recall = sum(recall_scores) / 50

print(f"Average Precision: {avg_precision:.5f}")
print(f"Average Recall: {avg_recall:.5f}")

Average Precision: 0.00100
Average Recall: 0.00222


In [13]:
# print("Top 10 documents similar to the query :")

#     similarity_score = cosine_similarities[0][index]
#     print(f"Document ID: {index+1}, Similarity Score: {similarity_score}")

Top 10 documents similar to the query :
Document ID: 270, Similarity Score: 0.4138472712343487
Document ID: 2428, Similarity Score: 0.332734839487535
Document ID: 605, Similarity Score: 0.304554049004568
Document ID: 60, Similarity Score: 0.30014259529274623
Document ID: 3, Similarity Score: 0.29565570697058774
Document ID: 1633, Similarity Score: 0.295380719381498
Document ID: 2121, Similarity Score: 0.2918489908692432
Document ID: 684, Similarity Score: 0.2803866282927753
Document ID: 2146, Similarity Score: 0.27088435472763067
Document ID: 864, Similarity Score: 0.26645478362330327
